In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline 

In [ ]:
raw_data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',sep=',', encoding='latin-1')

In [ ]:
raw_data.head()

In [ ]:
raw_data.columns


In [ ]:
raw_data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [ ]:
raw_data.head(4)

In [ ]:
df = raw_data.copy()

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.groupby('v1').describe().T

In [ ]:
df.rename(columns={"v1":"lable", "v2":"messages"}, inplace=True)

In [ ]:
df.head()

In [ ]:
df['length'] = df['messages'].apply(len)

In [ ]:
df.head()

In [ ]:
sns.set_style('darkgrid')
df['length'].plot.hist(bins=100, color='red')

In [ ]:
df['length'].describe()

In [ ]:
df[df['length'] == 910]['messages'].iloc[0]

In [ ]:
sns.set_style('darkgrid')
df.hist(column='length', by='lable', bins=100, figsize=(12,5))

In [ ]:
import string
from nltk.corpus import stopwords

In [ ]:
def remove_pucn(txt):
    """
    1. remove punctuation 
    2. remove stopwords 
    3. retrun clean text in list format
    """
    no_punc = [t for t in txt if t not in string.punctuation]
    no_punc = ''.join(no_punc)
    return [w for w in no_punc.split() if w.lower() not in stopwords.words('english') ]

In [ ]:
df.head(4)

In [ ]:
df['messages'].apply(remove_pucn)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bag_of_words_transf = CountVectorizer(analyzer=remove_pucn).fit(df['messages'])

In [ ]:
bag_of_words_transf.vocabulary_

In [ ]:
print(bag_of_words_transf.vocabulary_)

In [ ]:
msg50 = df['messages'][49]

In [ ]:
print(msg50)

In [ ]:
bow50 = bag_of_words_transf.transform([msg50])

In [ ]:
print(bow50)

In [ ]:
bow50.shape

In [ ]:
bag_of_words_transf.get_feature_names()[4777]

In [ ]:
features = bag_of_words_transf.get_feature_names()

In [ ]:
features = pd.DataFrame(features)

In [ ]:
type(features)

In [ ]:
features.head(3)

In [ ]:
msg_bow = bag_of_words_transf.transform(df['messages'])

In [ ]:
msg_bow.shape

In [ ]:
## Non Zero Messages
msg_bow.nnz

In [ ]:
sparsity = (100.0 * msg_bow.nnz / (msg_bow.shape[0] * msg_bow.shape[1]))
print('sparsity: {}'.format(sparsity))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tf_idf_trans = TfidfTransformer().fit(msg_bow)

In [ ]:
tfidf50 = tf_idf_trans.transform(bow50)

In [ ]:
print(tfidf50)

In [ ]:
tf_idf_trans.idf_[bag_of_words_transf.vocabulary_['actor']]

In [ ]:
msg_tf_idf = tf_idf_trans.transform(msg_bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB


In [ ]:
multiNB = MultinomialNB().fit(msg_tf_idf,df['lable'])


In [ ]:
multiNB.predict(tfidf50)

In [ ]:
gaussianNB = GaussianNB().fit(msg_tf_idf.toarray(), df['lable'])


In [ ]:
gaussianNB.predict(tfidf50.toarray())

In [ ]:
bernNB = BernoulliNB().fit(msg_tf_idf, df['lable'])

In [ ]:
bernNB.predict(bow50)

In [ ]:
multiN_all_pred = multiNB.predict(msg_tf_idf)

In [ ]:
gaussian_all_pred = gaussianNB.predict(msg_tf_idf.toarray())

In [ ]:
bernNB_all_pred = bernNB.predict(msg_tf_idf)

In [ ]:
print('Multinormial Naive Bayes all prediction')
print(multiN_all_pred)
print('\n'*2)

print('Gaussian Naive Bayes all prediction')
print(gaussian_all_pred)
print('\n'*2)

print('Bernoulli Naive Bayes all prediction')
print(bernNB_all_pred)
print('\n'*2)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
msg_train, msg_test, label_train, label_test = train_test_split(df['messages'], df['lable'], test_size =.30)

In [ ]:
msg_train.shape , msg_test.shape

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeLine = Pipeline([('bow', CountVectorizer(analyzer=remove_pucn)),
                     ('tfidf', TfidfTransformer()),
                     ('classifier', MultinomialNB())
                    ])

In [ ]:
pipeLine.fit(msg_train, label_train)

In [ ]:
pred = pipeLine.predict(msg_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(classification_report(label_test, pred))

In [ ]:
print(confusion_matrix(label_test, pred))

In [ ]:
print(accuracy_score(label_test, pred))